In [1]:
# Import + codice del prof: non serve personalizzarmi
import keras
from keras import layers
from keras import activations
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate, Normalization
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

def polar_generator(batchsize,grid=(10,10),noise=.002,flat=False):
  while True:
    x = np.random.rand(batchsize)
    y = np.random.rand(batchsize)
    out = np.zeros((batchsize,grid[0],grid[1]))
    xc = (x*grid[0]).astype(int)
    yc = (y*grid[1]).astype(int)
    for b in range(batchsize):
      out[b,xc[b],yc[b]] = 1
    #compute rho and theta and add some noise
    rho = np.sqrt(x**2+y**2) + np.random.normal(scale=noise)
    theta = np.arctan(y/np.maximum(x,.00001)) + np.random.normal(scale=noise)
    if flat:
      out = np.reshape(out,(batchsize,grid[0]*grid[1]))
    yield ((theta,rho),out)

In [3]:
# Iperparametri: personalizzami i valori e i nomi senza scasinare i risultati
n_train = 4000000
n_test = 20000
batch_size = 4096

In [4]:
# Popolazioni: personalizzami i nomi e rifattorizza
g1,g2 = 10,10
gen = polar_generator(n_train+n_test,grid=(g1,g2),noise=0.002,flat=True)
(theta,rho),y = next(gen)
x=np.array([i for i in zip(theta,rho)])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=n_test/(n_train+n_test), shuffle=True, random_state=1)

In [5]:
# Accuratezza: riscrivi in modo che calcoli la stessa cosa ma in modo diverso
def discretized_accuracy(true_maps, my_maps):
  equals = tf.equal(tf.argmax(true_maps, axis=1), tf.argmax(my_maps, axis=1))
  return tf.cast(tf.math.count_nonzero(equals), tf.float64) / tf.cast(len(true_maps), tf.float64)

In [6]:
# Rete: cambia i nomi delle variabili e la grandezza delle reti dense senza scasinare i risultati
theta_input = Input(shape=(1,))
theta_norm = Normalization(axis=None)
theta_norm.adapt(x_train[:,0])
theta_norm = theta_norm(theta_input)
theta_branch = Dense(2, activation=activations.softsign)(theta_norm)
theta_branch = Dense(4, activation=activations.tanh)(theta_branch)
theta_branch = Dense(4, activation=activations.sigmoid)(theta_branch)

rho_input = Input(shape=(1,))
rho_norm = Normalization(axis=None)
rho_norm.adapt(x_train[:,1])
rho_norm = rho_norm(rho_input)

rho_branch = Dense(4, activation=activations.softsign)(rho_norm)
rho_branch = Dense(4, activation=activations.swish)(rho_branch)
rho_branch = Dense(4, activation=activations.tanh)(rho_branch)
rho_branch = Dense(4, activation=activations.elu)(rho_branch)

concatenate_layer = Concatenate()([theta_branch, rho_branch])
output = layers.Dense(8, activation=keras.activations.swish)(concatenate_layer)
output = layers.Dense(16, activation=keras.activations.relu)(output)
output = layers.Dense(4, activation=keras.activations.gelu)(output)
output = layers.Dense(100, activation=activations.softmax)(output)

network = Model([theta_input, rho_input], output)
network.build((None, 2))
network.summary()
network.compile(
  optimizer=keras.optimizers.Adam(learning_rate=1e-3),
  loss=keras.losses.CategoricalCrossentropy(),
  metrics=['accuracy', discretized_accuracy]
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 normalization_1 (Normalization  (None, 1)           3           ['input_2[0][0]']                
 )                                                                                                
                                                                                                  
 normalization (Normalization)  (None, 1)            3           ['input_1[0][0]']            

In [8]:
# Prima della consegna, cancella tutto tranne:
# - la chiamata a fit
# - la prima chiamata a evaluate
# - la prima print del "Test Score"
# - la prima print dell'"Accuracy"
# Personalizza tutte le print.
load = input('wanna load previous weights?')
if load == 'y':
    network = keras.models.load_model("pesi")

while True:
    history = network.fit(
      x=[x_train[:,0], x_train[:,1]],
      y=y_train,
      epochs=15,
      batch_size=batch_size,
      validation_data=([x_test[:,0], x_test[:,1]], y_test),
      callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]
    )
    train = input('train again?')
    if train != 'y':
        break

score, _, acc  = network.evaluate([x_test[:,0], x_test[:,1]], y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Accuracy: {:.1f}%'.format(acc*100))

save = input('wanna load previous weights?')
if save == 'y':
    network.save("pesi")

while True:
    gen = polar_generator(20000,grid=(g1,g2),noise=0.002,flat=True)
    accs = 0.0
    lower = 0
    iters = 100
    for x in range(iters):
      (theta,rho),y = next(gen)
      x=np.array([i for i in zip(theta,rho)])

      score, _, acc = network.evaluate([x[:,0], x[:,1]], y, batch_size=batch_size)
      accs += acc
      if acc < 0.95:
        lower += 1

    print('------------------------')
    print('Accuracy: {:.1f}%'.format(accs/iters*100))
    print('Lower than 95%: {}/{}'.format(lower, iters))
    print('------------------------')

    test = input('test again?')
    if test != 'y':
        break

wanna load previous weights?n
Epoch 1/15
977/977 [==============================] - 18s 18ms/step - loss: 0.1181 - accuracy: 0.9607 - discretized_accuracy: 0.9607 - val_loss: 0.1170 - val_accuracy: 0.9610 - val_discretized_accuracy: 0.9609
Epoch 2/15
977/977 [==============================] - 17s 18ms/step - loss: 0.1140 - accuracy: 0.9622 - discretized_accuracy: 0.9622 - val_loss: 0.1134 - val_accuracy: 0.9619 - val_discretized_accuracy: 0.9619
Epoch 3/15
977/977 [==============================] - 18s 18ms/step - loss: 0.1116 - accuracy: 0.9623 - discretized_accuracy: 0.9623 - val_loss: 0.1103 - val_accuracy: 0.9614 - val_discretized_accuracy: 0.9614
Epoch 4/15
977/977 [==============================] - 17s 18ms/step - loss: 0.1082 - accuracy: 0.9636 - discretized_accuracy: 0.9636 - val_loss: 0.1055 - val_accuracy: 0.9635 - val_discretized_accuracy: 0.9635
Epoch 5/15
977/977 [==============================] - 19s 19ms/step - loss: 0.1057 - accuracy: 0.9641 - discretized_accuracy: 0.96